# Creación de las vistas materializadas para la capa ORO


### Vista dim_calendar
- c02_plata.sales_order (OrderDate)

In [14]:
CREATE MATERIALIZED LAKE VIEW c03_oro.dim_calendar AS 
WITH limits AS (
    SELECT 
        YEAR(MIN(so.OrderDate)) AS start_year,
        YEAR(MAX(so.OrderDate)) AS end_year
    FROM c02_plata.sales_order so
),
dates AS (
    SELECT explode(sequence(
        to_date(concat(start_year, '-01-01')),
        to_date(concat(end_year, '-12-31')),
        interval 1 day
    )) AS Date
    FROM limits
)
SELECT
    Date,
    YEAR(Date)  AS Year,
    MONTH(Date) AS Month,
    DAY(Date)   AS Day,
    DATE_FORMAT(Date, 'yyyy-MM') AS YearMonth,
    DATE_FORMAT(Date, 'EEEE')    AS WeekdayName,
    WEEKOFYEAR(Date) AS WeekOfYear,
    QUARTER(Date)    AS Quarter
FROM dates;


StatementMeta(, 91c78044-f413-4ba8-98a3-c99fc0fd6361, 15, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

### Vista dim_product
- Clave subrogada
- c02_plata.product 

In [1]:
CREATE MATERIALIZED LAKE VIEW c03_oro.dim_product AS 
SELECT
  ROW_NUMBER() OVER (ORDER BY ProductID) AS ProductKey,   -- clave subrogada
  ProductID,
  ProductName,
  ProductNumber,
  Color,
  Size,
  StandardCost,
  ListPrice,
  SubcategoryName,
  CategoryName
FROM c02_plata.product;

StatementMeta(, a056f742-bf62-41b5-bb8e-d266516fa6fc, 2, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

### Vista fact_sales_order
- Clave subrogada
- c02_plata.sales_order
- c03_oro.dim_product

In [2]:
CREATE MATERIALIZED LAKE VIEW c03_oro.fact_sales_order AS 
SELECT
  ROW_NUMBER() OVER (ORDER BY SalesOrderID, SalesOrderDetailID) AS SalesOrderKey, -- clave subrogada
  CAST(date_format(OrderDate,'yyyyMMdd') AS INT) AS DateKey,
  dp.ProductKey,
  s.SalesOrderDetailID,
  s.SalesOrderID,
  s.OrderDate,
  s.DueDate,
  s.ShipDate,
  s.ProductID,
  s.OrderQty,
  s.UnitPrice,
  s.UnitPriceDiscount,
  s.LineTotal,
  s.SubTotal,
  s.TaxAmt,
  s.Freight,
  s.TotalDue,
  s.CustomerID,
  s.SalesPersonID,
  s.TerritoryID,
  s.ShipMethodID,
  s.CurrencyRateID
FROM c02_plata.sales_order s
LEFT JOIN c03_oro.dim_product dp
  ON s.ProductID = dp.ProductID;


StatementMeta(, a056f742-bf62-41b5-bb8e-d266516fa6fc, 3, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>